#Fazendo leitura dos dados

In [1]:
!pip install pyspark==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 199 kB 43.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=60540522ad2173aa85a919b1a2584d20b8ad88dfcb2e3f903ec1c09c3eb0359c
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master('local[*]').appName("Transformacao com Spark").getOrCreate()

spark

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
dados = spark.read.parquet('/content/drive/MyDrive/Challenge_Data_Science_Alura/dados/dataset_parquet')

In [6]:
dados.show()

+--------------------+-----+---------+---------+-------+------+----+----------+------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+------------+----------+----------+--------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|condominio|  iptu|   valor|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|Zona Central|Zona Norte|Zona Oeste|Zona Sul|
+--------------------+-----+---------+---------+-------+------+----+----------+------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+------------+----------+----------+--------+
|03a386b6-7ab8-4ef...|    0|     43.0|        1|      2|     0|   1|     285.0|   0.0| 22999.0|       1|                 1|            1|                 1|       0|      0

#Preparação dos dados para os algoritmos do Spark MLlib

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
dados = dados.withColumnRenamed('valor','label')

In [9]:
X = []
for i in dados.columns:
  if i != 'id' and i != 'valor':
    X.append(i)
X

['andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'condominio',
 'iptu',
 'label',
 'Academia',
 'Animais permitidos',
 'Churrasqueira',
 'Condomínio fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'Portaria 24h',
 'Portão eletrônico',
 'Salão de festas',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul']

In [10]:
assembler = VectorAssembler(inputCols = X, outputCol = 'features')

In [11]:
dados_vetorizado = assembler.transform(dados).select('features', 'label')

In [14]:
dados_vetorizado.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------+--------+
|features                                                                                                   |label   |
+-----------------------------------------------------------------------------------------------------------+--------+
|[0.0,43.0,1.0,2.0,0.0,1.0,285.0,0.0,22999.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0]       |22999.0 |
|(23,[1,2,3,6,8,20],[44.0,1.0,2.0,170.0,110000.0,1.0])                                                      |110000.0|
|(23,[0,1,2,3,5,8,21],[4.0,55.0,1.0,2.0,1.0,115000.0,1.0])                                                  |115000.0|
|(23,[0,1,2,3,8,20],[2.0,55.0,1.0,2.0,115000.0,1.0])                                                        |115000.0|
|(23,[1,2,3,6,7,8,19],[50.0,1.0,1.0,1300.0,2679.0,125000.0,1.0])                                            |125000.0|
|(23,[0,1,2,3,5,6,8,10,11,12,14,16,18,21],[2.0,4

#Criação dos modelos de regressão

In [15]:
from pyspark.ml.regression import LinearRegression

In [16]:
treino, teste = dados_vetorizado.randomSplit([0.7, 0.3], seed=101) # é interessante definir uma seed de aleatoriedade para fazer comparação com resultados de outros testes

In [17]:
treino.count() 

46503

In [18]:
teste.count()

20048

In [19]:
lr = LinearRegression()

In [20]:
modelo_lr = lr.fit(treino)

In [21]:
previsoes_lr_treino = modelo_lr.transform(treino)
previsoes_lr_treino.show()

+--------------------+---------+------------------+
|            features|    label|        prediction|
+--------------------+---------+------------------+
|(23,[0,1,2,3,4,5,...| 580000.0| 579999.9999999986|
|(23,[0,1,2,3,4,5,...|3050000.0| 3049999.999999999|
|(23,[0,1,2,3,4,5,...|1590000.0|1589999.9999999984|
|(23,[0,1,2,3,4,5,...|1600000.0|1599999.9999999988|
|(23,[0,1,2,3,4,5,...| 390000.0|389999.99999999796|
|(23,[0,1,2,3,4,5,...| 917000.0| 916999.9999999987|
|(23,[0,1,2,3,4,5,...|1390000.0|1389999.9999999986|
|(23,[0,1,2,3,4,5,...|1050000.0|1049999.9999999984|
|(23,[0,1,2,3,4,5,...| 550000.0| 549999.9999999977|
|(23,[0,1,2,3,4,5,...|1120000.0|1119999.9999999984|
|(23,[0,1,2,3,4,5,...| 650000.0| 649999.9999999995|
|(23,[0,1,2,3,4,5,...| 640000.0| 639999.9999999988|
|(23,[0,1,2,3,4,5,...|1265000.0|1265000.0000000002|
|(23,[0,1,2,3,4,5,...| 906500.0| 906499.9999999987|
|(23,[0,1,2,3,4,5,...| 365000.0|364999.99999999854|
|(23,[0,1,2,3,4,5,...| 560000.0| 559999.9999999999|
|(23,[0,1,2,

#Avaliação dos modelos

In [22]:
resumo_treino = modelo_lr.summary

In [23]:
"""
R2 (ou **coeficiente de determinação**):
é uma medida estatística que representa a 
porcentagem de variação da variável resposta 
que é explicada pelo modelo de regressão. 
Por se tratar de uma porcentagem, seu valor 
varia de 0 a 1.
"""
resumo_treino.r2 

1.0

In [24]:
"""
RMSE (ou Raiz do Erro Quadrático Médio):
é a métrica que calcula a raiz quadrada do erro 
médio entre os valores reais e as predições da 
regressão. Quanto menor esse valor, melhor será 
o modelo, uma vez que estará cometendo menos erros.
"""
resumo_treino.rootMeanSquaredError

7.998662077689472e-10

In [25]:
resumo_teste = modelo_lr.evaluate(teste)

In [26]:
resumo_teste.r2

1.0

In [27]:
resumo_teste.rootMeanSquaredError

8.09400298219593e-10

In [28]:
print('Linear Regression')
print("="*30)
print("Dados de Treino")
print("="*30)
print("R²: %f" % resumo_treino.r2)
print("RMSE: %f" % resumo_treino.rootMeanSquaredError)
print("")
print("="*30)
print("Dados de Teste")
print("="*30)
print("R²: %f" % resumo_teste.r2)
print("RMSE: %f" % resumo_teste.rootMeanSquaredError)

Linear Regression
Dados de Treino
R²: 1.000000
RMSE: 0.000000

Dados de Teste
R²: 1.000000
RMSE: 0.000000
